<a href="https://colab.research.google.com/github/brownnyi/E-commerce_data-analyst/blob/main/bigquery_to_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install --upgrade google-cloud-bigquery

In [7]:
from google.cloud import bigquery

# If you don't specify credentials when constructing the client, the
# client library will look for credentials in the environment.
client = bigquery.Client()

In [8]:
from google.oauth2 import service_account
from google.cloud import bigquery

SERVICE_ACCOUNT_FILE = "bigquery-test-434505-a5eb5eebc9c9.json"  # 키 json 파일
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE)
project_id = "bigquery-test-434505" # 각자 프로젝트에 맞게 수정
client = bigquery.Client(credentials=credentials, project=project_id)

def import_bigquery_data(query):
    query_job = client.query(query)
    return query_job.to_dataframe()

data = import_bigquery_data('''
    SELECT *
    FROM `bigquery-public-data.google_trends.international_top_rising_terms`
    WHERE refresh_date = '2024-08-01'
    LIMIT 10
    ;
''')

data.head()

,country_code,term,score,rank,refresh_date,country_name,region_name,region_code,week,percent_gain
0,CH,arno kamminga,<NA>,11,2024-08-01,Switzerland,Aargau,CH-AG,2020-01-19,350
1,CH,arno kamminga,<NA>,11,2024-08-01,Switzerland,Aargau,CH-AG,2020-01-26,350
2,CH,arno kamminga,<NA>,11,2024-08-01,Switzerland,Aargau,CH-AG,2020-02-02,350
3,CH,arno kamminga,<NA>,11,2024-08-01,Switzerland,Aargau,CH-AG,2020-03-15,350
4,CH,arno kamminga,<NA>,11,2024-08-01,Switzerland,Aargau,CH-AG,2020-05-10,350


# 이전에 활용한 쿼리결과를 파이썬으로 추출해보기

1-0 필요한 데이터 미리 추출해주기
- 사용 테이블 : products, order_items, orders, users, events

In [9]:
import pandas as pd
import time

In [11]:
products = import_bigquery_data('''
    SELECT *
    FROM `bigquery-public-data.thelook_ecommerce.products`
    ;
''')
order_items = import_bigquery_data('''
    SELECT *
    FROM `bigquery-public-data.thelook_ecommerce.order_items`
    WHERE created_at BETWEEN '2024-06-01' AND '2024-07-01'
    ;
''')
orders = import_bigquery_data('''
    SELECT *
    FROM `bigquery-public-data.thelook_ecommerce.orders`
    WHERE created_at BETWEEN '2024-06-01' AND '2024-07-01'
    ;
''')
users = import_bigquery_data('''
    SELECT *
    FROM `bigquery-public-data.thelook_ecommerce.users`
    ;
''')
events = import_bigquery_data('''
    SELECT *
    FROM `bigquery-public-data.thelook_ecommerce.events`
    WHERE created_at BETWEEN '2024-06-01' AND '2024-07-01'
    ;
''')

In [13]:
products.head()

,id,cost,category,name,brand,retail_price,department,sku,distribution_center_id
0,13842,2.51875,Accessories,Low Profile Dyed Cotton Twill Cap - Navy W39S55D,MG,6.25,Women,EBD58B8A3F1D72F4206201DA62FB1204,1
1,13928,2.33835,Accessories,Low Profile Dyed Cotton Twill Cap - Putty W39S55D,MG,5.95,Women,2EAC42424D12436BDD6A5B8A88480CC3,1
2,14115,4.87956,Accessories,Enzyme Regular Solid Army Caps-Black W35S45D,MG,10.99,Women,EE364229B2791D1EF9355708EFF0BA34,1
3,14157,4.64877,Accessories,Enzyme Regular Solid Army Caps-Olive W35S45D (...,MG,10.99,Women,00BD13095D06C20B11A2993CA419D16B,1
4,14273,6.50793,Accessories,Washed Canvas Ivy Cap - Black W11S64C,MG,15.99,Women,F531DC20FDE20B7ADF3A73F52B71D0AF,1


## 예시쿼리1

In [16]:
data1 = import_bigquery_data('''
    SELECT B.product_id as product_id, A.name as product_name, A.category as product_category, COUNT(*) as num_of_orders
    FROM `bigquery-public-data.thelook_ecommerce.products` as A
    JOIN `bigquery-public-data.thelook_ecommerce.order_items` as B
    ON A.id = B.product_id
    GROUP BY 1,2,3
    ORDER BY num_of_orders DESC
''')

# 예시쿼리2

In [20]:
data2 = import_bigquery_data('''
    SELECT C.id as user_id, C.first_name, C.last_name, AVG(B.sale_price) as avg_sale_price
    FROM `bigquery-public-data.thelook_ecommerce.users` as C
    JOIN `bigquery-public-data.thelook_ecommerce.order_items` as B
    ON C.id = B.user_id
    GROUP BY 1,2,3
    ORDER BY avg_sale_price DESC
''')

# Q1. 2023년 가입 유저수

In [21]:
data3 = import_bigquery_data('''
    SELECT COUNT(id)
    FROM `bigquery-public-data.thelook_ecommerce.users`
    WHERE created_at BETWEEN '2023-01-01' AND '2023-12-31'
''')

data3.head()

,f0_
0,17024


In [23]:
users['created_at'] = [str(i)[:10] for i in users['created_at']]

In [25]:
users[(users['created_at'] >= '2023-06-01') & (users['created_at'] < '2024-01-01')]

,id,first_name,last_name,email,age,gender,state,street_address,postal_code,city,country,latitude,longitude,traffic_source,created_at
1,6971,Kendra,Smith,kendrasmith@example.net,56,F,Acre,3726 Sheila Estate Suite 980,69980-000,null,Brasil,-8.065346,-72.870949,Search,2023-11-15
13,71911,Zachary,Johnson,zacharyjohnson@example.com,33,M,Acre,3329 Tara Hills,69980-000,null,Brasil,-8.065346,-72.870949,Search,2023-07-21
16,96215,Gregory,Jones,gregoryjones@example.org,67,M,Acre,63499 Michelle Mills Apt. 993,69980-000,null,Brasil,-8.065346,-72.870949,Search,2023-09-20
40,34909,Christina,Jones,christinajones@example.net,13,F,Acre,438 Deanna Square,69917-400,Rio Branco,Brasil,-9.945568,-67.835610,Search,2023-08-17
51,87166,John,Miller,johnmiller@example.net,66,M,Acre,661 Berg Loop Suite 983,69917-400,Rio Branco,Brasil,-9.945568,-67.835610,Email,2023-07-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99951,39947,Dana,Mitchell,danamitchell@example.net,20,F,Île-de-France,09485 Zachary Forks Apt. 953,77220,Favières,France,48.744964,2.757914,Search,2023-09-16
99952,50950,Timothy,Gonzales,timothygonzales@example.org,17,M,Île-de-France,1063 Freeman Walk,77220,Favières,France,48.744964,2.757914,Facebook,2023-11-26
99981,69426,Pamela,Scott,pamelascott@example.org,15,F,Île-de-France,8820 Jessica Street Apt. 920,77580,Guérard,France,48.861376,2.951296,Display,2023-11-15
99984,9119,Jason,Mcbride,jasonmcbride@example.org,26,M,Île-de-France,0587 White Mall Suite 139,77130,La Grande-Paroisse,France,48.378520,2.963704,Search,2023-10-31


# Q2. 브라우저별 세션 수

In [28]:
events.head()

,id,user_id,sequence_number,session_id,created_at,ip_address,city,state,postal_code,browser,traffic_source,uri,event_type
0,1388597,<NA>,3,8bc22655-8b89-43ea-97e7-06941291e04e,2024-06-11 15:02:00+00:00,118.7.134.33,São Paulo,São Paulo,02675-031,Safari,Adwords,/cancel,cancel
1,1924848,<NA>,3,6f0a56b9-b3e6-4870-bb74-70e9c3a53a94,2024-06-18 09:48:00+00:00,83.166.57.243,Fuzhou,Inner Mongolia Autonomous Region,10051,Chrome,Organic,/cancel,cancel
2,1636406,<NA>,3,107f68c8-6694-4607-a095-7af8b8c7e7e3,2024-06-24 08:27:00+00:00,73.216.236.61,Shanghai,Beijing,101401,Chrome,Email,/cancel,cancel
3,1917556,<NA>,3,abb0047d-56db-42e2-88ec-69a0bb97a9d8,2024-06-28 18:35:00+00:00,155.199.5.6,Hefei,Liaoning,111003,Chrome,Email,/cancel,cancel
4,2053688,<NA>,3,97f8ad48-e07a-4c6c-8314-fc3ca3555d2f,2024-06-30 12:49:00+00:00,122.83.66.244,Myers Corner,New York,12590,Chrome,Adwords,/cancel,cancel


In [31]:
data4 = import_bigquery_data('''
    SELECT browser, COUNT(session_id) as total_session
    FROM `bigquery-public-data.thelook_ecommerce.events`
    WHERE sequence_number = 1 AND created_at BETWEEN '2023-06-01' AND '2024-01-01'
    GROUP BY browser
    ORDER BY total_session DESC
    ;
''')

In [32]:
data4.head()

,browser,total_session
0,Chrome,42138
1,Firefox,16938
2,Safari,16790
3,Other,4278
4,IE,4248


In [34]:
events['created_at'] = [str(i)[:10] for i in events['created_at']]

In [35]:
events.groupby('browser')[['session_id']].count().sort_values(by = 'session_id', ascending = False)

,session_id
browser,
Chrome,39571
Safari,15487
Firefox,15283
Other,3829
IE,3793


# 2023년 월별 매출액(sale_price * num_of_item), 주문수, 주문 유저수 계산

In [36]:
data5 = import_bigquery_data('''
    SELECT EXTRACT(MONTH FROM A.created_at) as month, SUM(A.sale_price * B.num_of_item) as revenue,
    COUNT(DISTINCT A.order_id) as order_count,
    COUNT(DISTINCT A.user_id) as customers_purchased
    FROM `bigquery-public-data.thelook_ecommerce.order_items` as A
    LEFT OUTER JOIN `bigquery-public-data.thelook_ecommerce.orders` as B
    ON A.order_id = B.order_id
    WHERE A.status NOT IN ('Cancelled', 'Returned') AND A.created_at BETWEEN '2024-06-01' AND '2025-01-01'
    GROUP BY EXTRACT(MONTH FROM A.created_at)
    ORDER BY month
    ;
''')

In [37]:
data5.head()

,month,revenue,order_count,customers_purchased
0,6,7.189570e+05,4448,4270
1,7,8.993938e+05,5379,5084
2,8,1.151198e+06,7579,6707
3,9,4.496905e+05,2483,1945


In [43]:
orders['created_at'] = [str(i)[:10] for i in orders['created_at']]
order_items['created_at'] = [str(i)[:10] for i in order_items['created_at']]

In [45]:
tmp_merge_data = pd.merge(
    order_items[['order_id','created_at','user_id','status','sale_price']],
    orders[['order_id','num_of_item']],
    how = 'left',
    on = 'order_id'
)

#조건절
tmp_merge_data = tmp_merge_data[~tmp_merge_data['status'].isin(['Cancelled','Returned'])]

tmp_merge_data['month'] = [i[5:7] for i in tmp_merge_data['created_at']]


result5 = tmp_merge_data.groupby('month').agg(
    revenue = ('sale_price',lambda x: (x * tmp_merge_data.loc[x.index, 'num_of_item']).sum()),
    order_count = ('order_id','nunique'),
    customers_purchased = ('user_id','nunique')
).reset_index()

In [42]:
tmp_merge_data.head()

,order_id,created_at,user_id,status,sale_price,num_of_item
0,104499,2024-06-18 15:17:44+00:00,84069,Shipped,1.50,1
1,87338,2024-06-24 00:01:11+00:00,70361,Returned,1.82,1
2,1606,2024-06-30 04:15:16+00:00,1336,Cancelled,1.95,1
3,121133,2024-06-01 13:43:56+00:00,97242,Complete,1.95,1
4,11056,2024-06-02 01:45:28+00:00,8900,Complete,2.95,1


In [46]:
result5

,month,revenue,order_count,customers_purchased
0,06,682267.470976,4448,4270


# Q4 order_items에서 status가 Cancelled, Returned된 상품드르이 남녀로 나누어 매출액(sale_price * num_of_item), 수량 계산 하기

In [48]:
data6 = import_bigquery_data('''
    SELECT B.gender, SUM(A.sale_price * B.num_of_item) as revenue,
    SUM(B.num_of_item) quantity
    FROM `bigquery-public-data.thelook_ecommerce.order_items` as A
    LEFT JOIN `bigquery-public-data.thelook_ecommerce.orders` as B
    ON A.order_id = B.order_id
    WHERE A.status IN ('Cancelled','Returned')
    GROUP BY B.gender
    ORDER BY revenue
    ;
''')

In [49]:
data6

,gender,revenue,quantity
0,F,2.379654e+06,42644
1,M,2.710904e+06,42486


# Q5. TRAFFIC SOURCE 별 전체 session, 구매 sessions, 구매 sessions/전체 sessions(세션별 구매율)

In [50]:
data7 = import_bigquery_data('''
    WITH a as (
        SELECT traffic_source, COUNT(session_id) as total_session
        FROM `bigquery-public-data.thelook_ecommerce.events`
        WHERE sequence_number = 1 AND created_at BETWEEN '2024-06-01' AND '2025-01-01'
        GROUP BY traffic_source
    ), b as (
        SELECT traffic_source, COUNT(session_id) as purchase_session
        FROM `bigquery-public-data.thelook_ecommerce.events`
        WHERE event_type = 'purchase' AND session_id IN (
            SELECT session_id
            FROM `bigquery-public-data.thelook_ecommerce.events`
            GROUP BY session_id
            HAVING MIN(created_at) BETWEEN '2024-06-01' AND '2025-01-01')
        GROUP BY traffic_source
    )

    SELECT a.traffic_source, a.total_session, b.purchase_session, ROUND(SAFE_DIVIDE(b.purchase_session, a.total_session), 4) as conversion_rate
    FROM a
    JOIN b
    ON a.traffic_source = b.traffic_source
    ORDER BY purchase_session DESC
    ;
''')

In [51]:
data7

,traffic_source,total_session,purchase_session,conversion_rate
0,Email,26977,16808,0.6230
1,Adwords,17967,11210,0.6239
2,Facebook,6099,3744,0.6139
3,YouTube,5937,3699,0.6230
4,Organic,3064,1875,0.6119
